# Data Access Groups (DAGs)

::: {.content-hidden when-format="html"}

In [1]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [2]:
#| echo: false
#| output: false
# enables the %%R magic, not necessary if you've already done this
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

C:\Users\CMD3303\Miniconda3\envs\r_python_jl\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


### Project Setup

::: {.panel-tabset}

#### R

In [3]:
%%capture --no-display 
%%R
library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [4]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] '1.1.9005'


In [5]:
%%R
# Load API tokens from the json file
token <- jsonlite::fromJSON('./../../json_api_data.json')$dev_token$'309'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [6]:
import redcap
import json
import csv
import pandas as pd
import numpy as np
import requests
import tempfile

In [7]:
# Load API tokens from the json file
key = json.load(open('./../../json_api_data.json'))
token = key['dev_token']['309']
url = key['dev_url']

project = redcap.Project(url, token)

:::

:::

In addition to user roles for access control, DAGs control which records each user can access. Users assigned to a DAG can only access records assigned to that particular DAG, and are blinded to records outside of their group. DAGs are particularly useful in multi-site or multi-jurisdictional projects to restrict sites from viewing records from other sites.  

Users can be assigned to a DAG after being added to the project. Users can be in multiple DAGs. Users not assigned to any DAG have global access and can see all records in the project. 

Records are assigned to DAG's using the 'Record Status Dashboard' in REDCap.

::: {.panel-tabset}

#### R

Cannot be imported with REDCapR. *See @sec-import_limitations for more information.* 

#### Python

DAGs can be imported using `import_dags()` and assigned using `import_user_dag_assignment()`. If the API user is assigned to multiple DAGs, they can be switched between DAGs using `switch_dag()`. 

View the current DAGs by exporting them:

In [8]:
dags = project.export_dags(format_type='df')
dags

,data_access_group_name,unique_group_name,data_access_group_id
0,Full Access,full_access,2708
1,Limited Access,limited_access,2709


Create and import a new DAG:

In [9]:
new_dag = [{"data_access_group_name": "Test DAG", "unique_group_name": ""}]
project.import_dags(new_dag)

1

**Note:** the `unique_group_name` field must be left blank as this is auto-generated by REDCap from the `data_access_group_name`.

The newly created DAG can now be seen and assigned to.

In [10]:
dags = project.export_dags(format_type='df')
dags

,data_access_group_name,unique_group_name,data_access_group_id
0,Full Access,full_access,2708
1,Limited Access,limited_access,2709
2,Test DAG,test_dag,2721


In [11]:
dag_mapping = [{"username": 'alexey.gilman@doh.wa.gov', "redcap_data_access_group": "test_dag"}]
project.import_user_dag_assignment(dag_mapping)

1

**Note:** the `redcap_data_access_group` name when importing is the same as `unique_group_name` when exporting DAGs.

::: {.content-hidden when-format="html"}

In [12]:
project.delete_dags(["test_dag"])

1

:::

:::